In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing import image
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.models import Sequential, Model
from keras.regularizers import l2
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Conv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D,Input,concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

# Detect Retina Damage From OCT Images With CNN

In [ ]:
import os

## Introduction

Retinal optical coherence tomography (OCT) is an imaging technique used to capture high-resolution cross sections of the retinas of living patients. Approximately 30 million OCT scans are performed each year, and the analysis and interpretation of these images take up a significant amount of time (Swanson and Fujimoto, 2017).

Figure 2. Representative Optical Coherence Tomography Images and the Workflow Diagram [Kermany et al., 2018] http://www.cell.com/cell/fulltext/S0092-8674(18)30154-5

(A) (Far left) Choroidal neovascularization (CNV) with neovascular membrane (white arrowheads) and associated subretinal fluid (arrows). (Middle left) Diabetic macular edema (DME) with retinal-thickening-associated intraretinal fluid (arrows). (Middle right) Multiple drusen (arrowheads) present in early AMD. (Far right) Normal retina with preserved foveal contour and absence of any retinal fluid/edema.

The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (NORMAL, CNV, DME, DRUSEN). There are 84,495 X-Ray images (JPEG) and 4 categories (NORMAL, CNV, DME, DRUSEN).

Images are labeled as (disease)-(randomized patient ID)-(image number by this patient) and split into 4 directories: CNV, DME, DRUSEN, and NORMAL.

CNV (Choroidal Neovascularization): Choroidal neovascularization refers to the abnormal growth of new blood vessels beneath the retina. This condition is typically associated with diseases like age-related macular degeneration.

DME (Diabetic Macular Edema): Diabetic macular edema is the accumulation of fluid in the macula region of the retina due to diabetic retinopathy. It can lead to vision loss.

DRUSEN: Drusen are small yellowish or whitish deposits seen on the retina. They are considered early signs of diseases like age-related macular degeneration (AMD).

NORMAL: Normal retina images represent healthy retinal cross-sections without any pathological signs or abnormalities.

These terms play a significant role in the diagnosis and monitoring of retinal diseases in medical studies utilizing optical coherence tomography (OCT) imaging technology.


In [ ]:
# Main Folder Path
folder_path = "/kaggle/input/kermany2018/OCT2017 "

# Sub Folder Paths
train_dir = f"{folder_path}/train"
val_dir = f"{folder_path}/val"
test_dir = f"{folder_path}/test"

In [ ]:
os.listdir(folder_path)

In [ ]:
print(f"Train Directory: {os.listdir(train_dir)}")
print(f"Validation Directory: {os.listdir(test_dir)}")
print(f"Test Directory: {os.listdir(val_dir)}")

In [ ]:
normal_train_dir = os.path.join(train_dir, "NORMAL")
normal_train_files = os.listdir(normal_train_dir)[:30]

normal_train_files

In [ ]:
normal_train_files[17]

In [ ]:
# Image file path
image_file = "NORMAL/NORMAL-8869683-18.jpeg"
image_path = os.path.join(train_dir, image_file)

# Read and display the image
image = Image.open(image_path)
plt.imshow(image)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
# Specify the directory where the dataset is located
dataset_directory = train_dir

# Create a dictionary to store the counts of images for each class
image_counts = {"CNV": 0, "DME": 0, "DRUSEN": 0, "NORMAL": 0}

# Iterate through the dataset to count the number of images for each class
for class_name in image_counts.keys():
    class_directory = os.path.join(dataset_directory, class_name)
    image_counts[class_name] = len(os.listdir(class_directory))

# Plotting the graph
classes = list(image_counts.keys())
counts = list(image_counts.values())

fig, ax = plt.subplots()
bars = ax.bar(classes, counts, color=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])

# Display total counts above the bars
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 0.1, round(yval), ha='center', va='bottom')

plt.xlabel('Classes')
plt.ylabel('Number of Images')
plt.title('Number of Images in Train Classes')
plt.show()

In [ ]:

# Specify the directory where the dataset is located
dataset_directory = test_dir

# Create a dictionary to store the counts of images for each class
image_counts = {"CNV": 0, "DME": 0, "DRUSEN": 0, "NORMAL": 0}

# Iterate through the dataset to count the number of images for each class
for class_name in image_counts.keys():
    class_directory = os.path.join(dataset_directory, class_name)
    image_counts[class_name] = len(os.listdir(class_directory))

# Plotting the graph
classes = list(image_counts.keys())
counts = list(image_counts.values())

fig, ax = plt.subplots()
bars = ax.bar(classes, counts, color=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])

# Display total counts above the bars
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 0.1, round(yval), ha='center', va='bottom')

plt.xlabel('Classes')
plt.ylabel('Number of Images')
plt.title('Number of Images in Test Classes')
plt.show()

In [ ]:
# Specify the directory where the dataset is located
dataset_directory = train_dir

# Specify the classes
classes = ["CNV", "DME", "DRUSEN", "NORMAL"]

# Loop to return the shape of an image from each class
for class_name in classes:
    # Create the class directory
    class_directory = os.path.join(dataset_directory, class_name)

    # Select an image in the class directory
    image = os.listdir(class_directory)[55]

    # Load the image
    image_path = os.path.join(class_directory, image)
    image = cv2.imread(image_path)

    # Print the shape of the image
    print(f"Class: {class_name}, Image Shape: {image.shape}")

In [ ]:
# Specify the directory where the dataset is located
dataset_directory = train_dir

# Specify the classes
classes = ["CNV", "DME", "DRUSEN", "NORMAL"]

# Loop to return the shape of an image from each class
for class_name in classes:
    # Create the class directory
    class_directory = os.path.join(dataset_directory, class_name)

    # Select an image in the class directory
    image = os.listdir(class_directory)[55]

    # Load the image
    image_path = os.path.join(class_directory, image)
    image = cv2.imread(image_path)

    # Print the shape of the image
    print(f"Class: {class_name}, Image Shape: {image.shape}")

    x= []
    y = []

    img = image
    d1,d2,colors = img.shape
    x.append(d1)
    y.append(d2)

print(np.mean(x))
print(np.mean(y))



In [ ]:

# Specify the classes
classes = ["CNV", "DME", "DRUSEN", "NORMAL"]

# Create a Matplotlib figure and axes
fig, axs = plt.subplots(1, 4, figsize=(12, 3))

# Load and display one example from each class side by side
for i, class_name in enumerate(classes):
    image_path = os.path.join(train_dir, class_name, os.listdir(os.path.join(train_dir, class_name))[0])
    img = imread(image_path)
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(f" {class_name}")

plt.show()

# Model 1

## Creating the Model

In [ ]:
batch_size = 32

In [ ]:
image_shape = (299,299,1)

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.20, # Shift the pic width by a max of 20%
                               height_shift_range=0.15, # Shift the pic height by a max of 15%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.15, # Shear means cutting away part of the image (max 15%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
image_gen.flow_from_directory(train_dir)

In [ ]:
image_gen.flow_from_directory(test_dir)

In [ ]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=image_shape, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dense(4, activation='softmax'))


In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_dir,
                                               target_size=(299,299),
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='categorical', shuffle=True)

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_dir,
                                               target_size=(299,299),
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='categorical', shuffle=False)

In [ ]:
results = model.fit(train_image_gen,epochs=10, validation_data=test_image_gen, callbacks=[early_stop])

In [ ]:
summary = pd.DataFrame(model.history.history)
summary.head()

In [ ]:
model.history.history

In [ ]:
history_df = pd.DataFrame(model.history.history)
history_df.index.name = 'Epoch'
history_df.index += 1  # Epoch numaralarını 1'den başlat

# DataFrame'i yazdır
print(history_df)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.loss, label="loss")
plt.plot(summary.val_loss, label="val_loss")
plt.legend(loc="upper right")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.accuracy, label="accuracy")
plt.plot(summary.val_accuracy, label="val_accuracy")
plt.legend(loc="upper left")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

In [ ]:
model.evaluate(test_image_gen)

In [ ]:
model.metrics_names

In [ ]:
pred_probabilities = model.predict(test_image_gen)

In [ ]:
pred_probabilities

In [ ]:
test_image_gen.classes

In [ ]:
predictions = pred_probabilities

In [ ]:
predictions_argmax = np.argmax(predictions, axis=1)

print(classification_report(test_image_gen.classes, predictions_argmax))

In [ ]:
model.save('CNN_model1.h5')

# INCEPTİONV3

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.20, # Shift the pic width by a max of 20%
                               height_shift_range=0.15, # Shift the pic height by a max of 15%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.15, # Shear means cutting away part of the image (max 15%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )
batch_size = 32

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_dir,
                                               target_size=(299,299),
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='categorical', shuffle=True)

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_dir,
                                               target_size=(299,299),
                                        
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='categorical', shuffle=False)

In [ ]:
# Define input shape
input_shape = (299, 299, 1)  # Grayscale images with shape 299x299

# Create the input layer
inputs = Input(shape=input_shape)

# Convert grayscale to RGB by repeating the single channel across three channels
x = tf.keras.layers.Concatenate()([inputs, inputs, inputs])

# Load the InceptionV3 model with ImageNet weights, without the top layers, and set the input tensor
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=x)

# Flatten the output of the base model
x = Flatten()(base_model.output)

# Fully connected layers
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(64, activation='relu')(x)

# Output layer
outputs = Dense(4, activation='softmax')(x)

# Create the model
InceptionV3_model = Model(inputs=inputs, outputs=outputs)

# Optimizer
optimizer = Adam(learning_rate=0.0001)

# Compile the model
InceptionV3_model.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])

# Print the model summary
InceptionV3_model.summary()



In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience = 4)
checkpoint = ModelCheckpoint('InceptionV3_tuning.keras',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')


In [ ]:
InceptionV3_model_results = InceptionV3_model.fit(train_image_gen,epochs=10,
                    validation_data=test_image_gen,
                    callbacks=[early_stop, checkpoint])

In [ ]:
summary = pd.DataFrame(InceptionV3_model.history.history)
summary

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.loss, label="loss")
plt.plot(summary.val_loss, label="val_loss")
plt.legend(loc="upper right")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.accuracy, label="accuracy")
plt.plot(summary.val_accuracy, label="val_accuracy")
plt.legend(loc="upper left")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

In [ ]:
pred_probabilities = InceptionV3_model.predict(test_image_gen)

In [ ]:
predictions_argmax = np.argmax(pred_probabilities, axis=1)

print(classification_report(test_image_gen.classes, predictions_argmax))

In [ ]:
conf_matrix = confusion_matrix(test_image_gen.classes, predictions_argmax)
print(conf_matrix)


plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# InceptionV3 Fine-Tuning Model

In [ ]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.20, # Shift the pic width by a max of 20%
                               height_shift_range=0.15, # Shift the pic height by a max of 15%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.15, # Shear means cutting away part of the image (max 15%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )
batch_size = 32

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_dir,
                                               target_size=(299,299),
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='categorical', shuffle=True)

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_dir,
                                               target_size=(299,299),
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='categorical', shuffle=False)

In [ ]:
# Define input shape
input_shape = (299, 299, 1)  # Grayscale images with shape 299x299

# Create the input layer
inputs = Input(shape=input_shape)

# Convert grayscale to RGB by repeating the single channel across three channels
x = tf.keras.layers.Concatenate()([inputs, inputs, inputs])

# Load the InceptionV3 model with ImageNet weights, without the top layers, and set the input tensor
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=x)

# Flatten the output of the base model
x = Flatten()(base_model.output)

# Fully connected layers with L2 regularization
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Increased dropout rate

x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Increased dropout rate

x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)

# Output layer
outputs = Dense(4, activation='softmax')(x)

# Create the model
InceptionV3_model = Model(inputs=inputs, outputs=outputs)

# Optimizer
optimizer = Adam(learning_rate=1e-5)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7)

checkpoint = ModelCheckpoint('InceptionV3_tuning.keras',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')


# Compile the model
InceptionV3_model.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])

# Print the model summary
InceptionV3_model.summary()

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
InceptionV3_model_results = InceptionV3_model.fit(train_image_gen,epochs=20,
                    validation_data=test_image_gen,
                    callbacks=[early_stopping, checkpoint])

In [ ]:
summary = pd.DataFrame(InceptionV3_model.history.history)
summary

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.loss, label="loss")
plt.plot(summary.val_loss, label="val_loss")
plt.legend(loc="upper right")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(summary.accuracy, label="accuracy")
plt.plot(summary.val_accuracy, label="val_accuracy")
plt.legend(loc="upper left")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

In [ ]:
pred_probabilities = InceptionV3_model.predict(test_image_gen)

In [ ]:
predictions_argmax = np.argmax(pred_probabilities, axis=1)

print(classification_report(test_image_gen.classes, predictions_argmax))

In [ ]:
conf_matrix = confusion_matrix(test_image_gen.classes, predictions_argmax)
print(conf_matrix)


plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

#  Evaluating Model Performance on Unseen Validation Data


In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_gen = image_gen.flow_from_directory(val_dir,
                                               target_size=(299,299),
                                               color_mode='grayscale',
                                               batch_size=batch_size,
                                               class_mode='categorical', shuffle=False)

In [ ]:
val_gen.reset()
val_probabilities = InceptionV3_model.predict(val_gen)
val_predictions_argmax = np.argmax(val_probabilities, axis=1)

In [ ]:
print(classification_report(val_gen.classes, val_predictions_argmax))